In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
from ROOT import TChain

In [ ]:
def open_df(fname,treename="flashtrigger/opflash_opflash_tree"):
    return pd.DataFrame( root2array(fname, treename) )

In [ ]:
def get_eff(df,param,xmin,xmax,interval,total):
    if total <= 0:
        print 'Cannot calculate efficiency w/o total <= 0!'
        raise Exception
    underflow = float(len(df.query('%s <=0' % param).groupby(['run','subrun','event'])))
    xaxis = np.arange(xmin,xmax,interval)

    eff = np.zeros(len(xaxis))
    for index in xrange(len(xaxis)):
        cut_value = xaxis[index]
        ctr = 0
        ctr = underflow + float(len(df.query('%s > %g' % (param,cut_value)).groupby(['run','subrun','event'])))
        eff[index] = ctr / total

    return (xaxis,eff)

In [ ]:
def plot_eff(df,param,xmin,xmax,interval,total):
    
    p='pe_total'
    xmin=0
    xmax=1000
    interval=5
    total=82875
    
    sg_xaxis,sg_eff = get_eff(df,p,xmin,xmax,interval,total)
    #sn = np.zeros(len(sg_xaxis))
    #for index in xrange(len(sn)):
    #    sn[index] = np.sqrt(sg_eff[index]/(1.-bg_eff[index]))
    #    print sg_xaxis[index],sg_eff[index],bg_eff[index],sn[index]
    plt.figure(figsize=(12,8))
    plt.ylim(-0.00,0.01)
    plt.xlim(xmin-interval*2,xmax+interval*2)
    plt.plot(sg_xaxis, sg_eff, marker='o', linestyle='--',color='blue')
    #plt.plot(sg_xaxis, sn,     marker='o', linestyle='--',color='orange')
    plt.xlabel('%s [p.e.]' % p,fontsize=20)
    plt.ylabel('Efficiency',fontsize=20)
    plt.tick_params(labelsize=20)
    plt.grid()
    plt.title('Total Photo-Electron (p.e.) Cut Selection Efficiency',fontsize=20)
    plt.show()
    
    if (xmax - xmin)/interval > 100:
        plt.figure(figsize=(12,8))
        plt.ylim(-0.00,0.01)
        plt.xlim(xmin,xmin + interval * 50)
        plt.plot(sg_xaxis, sg_eff, marker='o', linestyle='--',color='blue')
        #plt.plot(sg_xaxis, sn,     marker='o', linestyle='--',color='orange')
        plt.xlabel('%s [p.e.]' % p,fontsize=20)
        plt.ylabel('Efficiency',fontsize=20)
        plt.tick_params(labelsize=20)
        plt.grid()
        plt.title('Total Photo-Electron (p.e.) Cut Selection Efficiency',fontsize=20)
        plt.show()
        
    sg_of = len(df.query('%s > %g' % (p,xmax))) / float(total) * 100
    sg_of = int(sg_of*1000.)/1000.
        
    print 'overflow',  sg_of,'[%]'

In [ ]:
#
# Data loading
#
df = open_df('cosmic.root')

In [ ]:
#
# Making efficiency plot as a function of PE cut
#
beam_candidates=df.query('-0.1<dt and dt<1.7')
total = len(df.groupby(['run','subrun','event']))
plot_eff(df=beam_candidates, param='pe_total', xmin=0, xmax=1000, interval=5, total=82875)


In [ ]:
#
# Flash Timing distribution 
#
t_cand = df.query('pe_total>50')
fig, ax = plt.subplots(figsize=(12, 8))
t_cand['dt'].hist(ax=ax,bins=200)
plt.xlabel('Time w.r.t. Trigger [us]',fontsize=20)
plt.ylabel('Flash Count',fontsize=20)
plt.title('Flash Time Distribution',fontsize=20)
ax.tick_params(labelsize=20)